In [2]:
in_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_files/labels"

out_path = "C:/Users/RAZER/Downloads/Task09_Spleen/dicm_groups/labels"

In [4]:
!pip install glob2    # for revealing the path

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9311 sha256=7500d64a21ff0b500b4b15b74048c2e70dab318c850b0ac55bd26f9ad433a95d
  Stored in directory: c:\users\razer\appdata\local\pip\cache\wheels\95\59\ed\52469ecfae47a78c25f11bcb49db42ff84629df01861876883
Successfully built glob2


In [5]:
!pip install pytest-shutil                    #to move the files 

ERROR: Invalid requirement: '#to'
